In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

try:
    cpus = mp.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

pool = mp.Pool(processes=cpus)

def next_training_batch(n):
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

In [6]:
rows, cols = dimensions
size = rows*cols

In [7]:
model = tf.keras.models.Sequential()

In [8]:
model.add(tf.keras.layers.Dense(units=size, input_dim=size*10))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Reshape((rows, cols, 1)))

model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Reshape((4*4*32,)))

#model.add(tf.keras.layers.Dense(units=1024))
#model.add(tf.keras.layers.Activation('relu'))

#model.add(tf.keras.layers.Dropout(rate=0.5))

model.add(tf.keras.layers.Dense(units=size*2))
model.add(tf.keras.layers.Activation('softmax'))

In [9]:
model.compile(optimizer='adam', loss='poisson')

In [10]:
print(model.metrics_names)

['loss']


In [11]:
savePath = "/media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines8"

In [ ]:
for iteration in range(1000001):
    x_batch, y_batch, _ = next_training_batch(10000)
    loss = model.train_on_batch(np.array(x_batch), np.array(y_batch))
    print('{0}: Loss at step {1}: {2}'.format(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    np.save('{0}/training_data/x_batch-{1}.npy'.format(savePath, iteration), np.array(x_batch))
    np.save('{0}/training_data/y_batch-{1}.npy'.format(savePath, iteration), np.array(y_batch))
    if iteration % 500 == 0:
        model.save('{0}/model-{1}.h5'.format(savePath, iteration))

2017-11-10 19:24:28: Loss at step 0: 0.04574406519532204
2017-11-10 19:24:30: Loss at step 1: 0.04570643976330757
2017-11-10 19:24:32: Loss at step 2: 0.045675598084926605
2017-11-10 19:24:34: Loss at step 3: 0.04564366489648819
2017-11-10 19:24:36: Loss at step 4: 0.045610737055540085
2017-11-10 19:24:38: Loss at step 5: 0.04557596519589424
2017-11-10 19:24:40: Loss at step 6: 0.045546479523181915
2017-11-10 19:24:42: Loss at step 7: 0.04550795257091522
2017-11-10 19:24:45: Loss at step 8: 0.045479610562324524
2017-11-10 19:24:47: Loss at step 9: 0.04544918239116669
2017-11-10 19:24:49: Loss at step 10: 0.045431364327669144
2017-11-10 19:24:51: Loss at step 11: 0.04542260617017746
2017-11-10 19:24:53: Loss at step 12: 0.04540497437119484
2017-11-10 19:24:55: Loss at step 13: 0.04537178948521614
2017-11-10 19:24:57: Loss at step 14: 0.04534301534295082
2017-11-10 19:24:59: Loss at step 15: 0.045312389731407166
2017-11-10 19:25:01: Loss at step 16: 0.045268259942531586
2017-11-10 19:25: